In [16]:
import pandas as pd
from geopy.distance import geodesic

def find_nearby_rows(df, distance_threshold=6):
    """
    각 행의 위도 및 경도를 비교하고, 거리가 지정된 임계값(km) 이내인 행을 찾는 함수.

    Args:
        df (pd.DataFrame): 위도, 경도 및 ID 열을 포함하는 Pandas DataFrame.
        distance_threshold (float): km 단위의 거리 임계값 (기본값: 6).

    Returns:
        pd.DataFrame: 각 행에 대해 거리가 임계값 이내인 다른 행의 ID를 포함하는 DataFrame.
    """

    n_rows = len(df)
    nearby_ids = []

    for i in range(n_rows):
        row1 = df.iloc[i]
        lat1, lon1 = row1['Latitude'], row1['Longitude']
        nearby_rows_for_i = []

        for j in range(n_rows):
            if i == j:  # 동일한 행은 비교하지 X
                continue

            row2 = df.iloc[j]
            lat2, lon2 = row2['Latitude'], row2['Longitude']

            # geopy 라이브러리의 geodesic 함수를 사용, 두 지점 간의 거리를 계산
            distance = geodesic((lat1, lon1), (lat2, lon2)).km

            if distance <= distance_threshold:
                nearby_rows_for_i.append(row2['ID'])

        nearby_ids.append(nearby_rows_for_i)

    # result df 생성
    result_df = pd.DataFrame(index=df.index)

    for k in range(len(nearby_ids)):
      for l in range(len(nearby_ids[k])):
        result_df.loc[k, f'nearby_id_{l+1}'] = nearby_ids[k][l]

    return result_df


# 입력데이터
df = pd.read_csv("Apis_cerana_result.csv")


result = find_nearby_rows(df)
print(result)

      nearby_id_1  nearby_id_2  nearby_id_3  nearby_id_4  nearby_id_5  \
0             2.0         22.0        162.0        407.0        782.0   
1             1.0          NaN          NaN          NaN          NaN   
2            81.0         83.0        165.0        272.0        325.0   
3            15.0         80.0        102.0        136.0        211.0   
4            22.0        162.0        407.0        782.0       1078.0   
...           ...          ...          ...          ...          ...   
1154        144.0          NaN          NaN          NaN          NaN   
1155          NaN          NaN          NaN          NaN          NaN   
1156          3.0         81.0         83.0        165.0        272.0   
1157          NaN          NaN          NaN          NaN          NaN   
1158          1.0          5.0         22.0        162.0        407.0   

      nearby_id_6  nearby_id_7  nearby_id_8  nearby_id_9  nearby_id_10  ...  \
0          1078.0       1159.0          NaN 

In [18]:
result.to_csv("nearby_id_under6km_cer.csv")

In [9]:
df1 = pd.read_csv("Apis_mellifera_result.csv")


result1 = find_nearby_rows(df1)
print(result1)

     nearby_id_1  nearby_id_2  nearby_id_3  nearby_id_4  nearby_id_5
0            NaN          NaN          NaN          NaN          NaN
1            NaN          NaN          NaN          NaN          NaN
2            NaN          NaN          NaN          NaN          NaN
3            NaN          NaN          NaN          NaN          NaN
4           17.0          NaN          NaN          NaN          NaN
..           ...          ...          ...          ...          ...
155        117.0        155.0          NaN          NaN          NaN
156          NaN          NaN          NaN          NaN          NaN
157          NaN          NaN          NaN          NaN          NaN
158          NaN          NaN          NaN          NaN          NaN
159          NaN          NaN          NaN          NaN          NaN

[160 rows x 5 columns]


In [10]:
result1.to_csv("nearby_id_under6km_mel.csv")

In [19]:
links = []
for i in range(len(df)):
    for j in range(1, len(result.columns) + 1):  # nearby_id 열 순회
        nearby_id = result.loc[i, f'nearby_id_{j}']
        if pd.notna(nearby_id):  # NaN 값이 아닌 경우에만 링크 생성
            links.append({'Source': df.loc[i, 'ID'], 'Target': nearby_id})

links_df = pd.DataFrame(links)
print("\n링크 데이터:\n", links_df)


링크 데이터:
       Source  Target
0          1     2.0
1          1    22.0
2          1   162.0
3          1   407.0
4          1   782.0
...      ...     ...
8497    1159    22.0
8498    1159   162.0
8499    1159   407.0
8500    1159   782.0
8501    1159  1078.0

[8502 rows x 2 columns]


In [20]:
links_df.to_csv("links_cer.csv")

In [11]:
links1 = []
for i in range(len(df1)):
    for j in range(1, len(result1.columns) + 1):  # nearby_id 열 순회
        nearby_id1 = result1.loc[i, f'nearby_id_{j}']
        if pd.notna(nearby_id1):  # NaN 값이 아닌 경우에만 링크 생성
            links1.append({'Source': df1.loc[i, 'ID'], 'Target': nearby_id1})

links_df1 = pd.DataFrame(links1)
print("\n링크 데이터:\n", links_df1)


링크 데이터:
      Source  Target
0         5    17.0
1         6     7.0
2         6    23.0
3         7     6.0
4         7    23.0
..      ...     ...
147     150   117.0
148     155   117.0
149     155   156.0
150     156   117.0
151     156   155.0

[152 rows x 2 columns]


In [21]:
links_df1.to_csv("links_mel.csv")